In [1]:
#Importing needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
import re
import datetime

####Importing geopsatial libraries
import fiona
from shapely.geometry import shape, mapping, Point, Polygon
from shapely import wkt
import rtree
import fiona.crs
import geopandas as gpd
import pyproj


#### Importing Raster Libraries
import rasterio
import rioxarray as rxr
from rasterio.plot import plotting_extent
import rasterstats as rs

####Ignoring Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#### Load the facilities data
facilities_data = gpd.read_file('facilities_points_shp.shp')
facilities_data.head()

,id,FACID,CAPACITY,LONGITUDE,LATITUDE,geometry
0,1,30000037,158,-121.416183,38.463381,POINT (-121.41618 38.46338)
1,2,30000108,52,-120.764691,38.350710,POINT (-120.76469 38.35071)
2,3,30000109,64,-121.100445,38.944440,POINT (-121.10044 38.94444)
3,4,30000113,646,-121.456678,38.555181,POINT (-121.45668 38.55518)
4,5,30000114,63,-119.997590,38.911599,POINT (-119.99759 38.91160)


In [3]:
#### Rendering the raster dataset

import rasterio
from rasterio.features import shapes
mask = None
with rasterio.Env():
    with rasterio.open('/Users/akashyadav/Desktop/facilities_risk/dummy_raster_2.tif') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

In [4]:
#### Creating a list of pixel values
geoms = list(results)

In [5]:
#### Visualising 
geoms[10000]

{'properties': {'raster_val': 3.0},
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-122.70909276956618, 41.954306727310005),
    (-122.70909276956618, 41.95399096542483),
    (-122.70877700768101, 41.95399096542483),
    (-122.70877700768101, 41.954306727310005),
    (-122.70909276956618, 41.954306727310005)]]}}

In [6]:
#### Vectorizing the raster data with a CRS configuration of 4326
gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
gpd_polygonized_raster.crs = "EPSG:4326"

In [8]:
#### Saving the file
gpd_polygonized_raster.to_file('calfire_ftz_polygonized_raster.geojson', driver='GeoJSON')

In [144]:
#### Subsetting the vectorized data to only get values greater than 3
gpd_polygonized_raster_subset = gpd_polygonized_raster[gpd_polygonized_raster['raster_val']>3]
gpd_polygonized_raster_subset.head()

,geometry,raster_val
8861,POINT (-123.06827 41.95889),5.0
8920,POINT (-123.04045 41.95870),5.0
8952,POINT (-123.06599 41.95783),5.0
8986,POINT (-123.06880 41.95846),5.0
8987,POINT (-123.05748 41.95771),5.0


In [146]:
gdp_centroids = gpd_polygonized_raster_subset

In [147]:
gdp_centroids.geometry = gdp_sub.geometry.centroid
gdp_centroids.head()

,geometry,raster_val
8861,POINT (-123.06827 41.95889),5.0
8920,POINT (-123.04045 41.95870),5.0
8952,POINT (-123.06599 41.95783),5.0
8986,POINT (-123.06880 41.95846),5.0
8987,POINT (-123.05748 41.95771),5.0


In [1]:
#### Functions for getting nearest distance to FTZ pixel 
#### Reference: https://autogis-site.readthedocs.io/en/2019/notebooks/L3/nearest-neighbor-faster.html 

from sklearn.neighbors import BallTree

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name
    
    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]
    
    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius
        
        return closest_points

In [150]:
#### Finding the closest pixels
closest_pixels = nearest_neighbor(facilities_data, gdp_centroids, return_dist=True)

In [151]:
#### Converting distance to miles
closest_pixels['distance_miles'] = closest_pixels['distance']*0.000621371
closest_pixels.head()

,geometry,raster_val,distance,distance_miles
0,POINT (-121.17244 39.48615),5.0,64993.714407,40.385209
1,POINT (-120.84783 38.03081),5.0,20429.099877,12.694050
2,POINT (-121.16044 39.48552),5.0,31812.132070,19.767136
3,POINT (-121.17244 39.48615),5.0,62398.647408,38.772710
4,POINT (-120.09316 38.15916),5.0,43218.062069,26.854450


In [152]:
#### Joining raster value, distance and distance in miles columns back to the original dataframe
facilities_data['raster_val'] = closest_pixels['raster_val']
facilities_data['distance'] = closest_pixels['distance']
facilities_data['distance_miles'] = closest_pixels['distance_miles']

In [153]:
facilities_data.head()

,id,FACID,CAPACITY,LONGITUDE,LATITUDE,geometry,raster_val,distance,distance_miles
0,1,30000037,158,-121.416183,38.463381,POINT (-121.41618 38.46338),5.0,64993.714407,40.385209
1,2,30000108,52,-120.764691,38.350710,POINT (-120.76469 38.35071),5.0,20429.099877,12.694050
2,3,30000109,64,-121.100445,38.944440,POINT (-121.10044 38.94444),5.0,31812.132070,19.767136
3,4,30000113,646,-121.456678,38.555181,POINT (-121.45668 38.55518),5.0,62398.647408,38.772710
4,5,30000114,63,-119.997590,38.911599,POINT (-119.99759 38.91160),5.0,43218.062069,26.854450
